In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from datetime import datetime
from app.table_filler import TableFiller
from app.date_formatter import DateFormatter

from_date = datetime(2025, 3, 1)
to_date = datetime(2025, 3, 26)
SPREADSHEET_NAME = "rnp-experiment"
WORKSHEET_NAME = "task"

report_dates = DateFormatter.generate_date_range(from_date, to_date)
table_filler = TableFiller()

for report_date in report_dates:
    table_filler.fill_sheet_column(SPREADSHEET_NAME, WORKSHEET_NAME, report_date)

TypeError: strptime() argument 1 must be str, not datetime.datetime